In [1]:
import os
import sys
import math
import pickle
import logging
from pathlib import Path

import scipy as sp

from sklearn.linear_model import LinearRegression

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)
_logger = logging.getLogger()

In [2]:
from bhm_at_scale.handler import ModelHandler
from bhm_at_scale.model import model, guide, local_guide, check_model_guide, predictive_model, Site
from bhm_at_scale.utils import summary, stats_to_df, preds_to_df

In [3]:
import jax.numpy as jnp
from jax import random, ops
from jax import lax
from jax import jit
from jax.numpy import DeviceArray
import numpy as np
import numpyro
from numpyro import optim
import numpyro.distributions as dist
from numpyro.infer import ELBO, SVI, Predictive
from numpyro.infer.svi import SVIState

In [4]:
X_train = jnp.array(np.load('../data/preprocessed/X_train.npz')['arr_0'])
X_train.shape

(1000, 942, 24)

## Fit the hierachical model

In [5]:
check_model_guide(X_train, model=model, guide=guide)
train_handler = ModelHandler(model=model, guide=guide)

In [6]:
train_handler.fit(X_train, n_epochs=5_000, log_freq=1_000, lr=0.1)

epoch:    0 loss:      114879.0703
epoch: 1000 loss:        6734.7920
epoch: 2000 loss:        6423.1851
epoch: 3000 loss:        6406.2021
epoch: 4000 loss:        6383.2476
epoch: 5000 loss:        6394.3120


6371.9970703125

In [7]:
train_handler.fit(X_train, n_epochs=1_000, log_freq=200, lr=0.001)

epoch:    0 loss:        6371.9971
epoch:  200 loss:        6369.5137
epoch:  400 loss:        6368.5181
epoch:  600 loss:        6368.1230
epoch:  800 loss:        6370.3921
epoch: 1000 loss:        6370.7759


6367.5048828125

## Checkpoint: Save/restore current state

In [8]:
with open('../data/result/optim_state.pickle', 'bw') as fh:
    train_handler.dump_optim_state(fh)

In [9]:
train_handler = ModelHandler(model=model, guide=guide)
with open('../data/result/optim_state.pickle', 'br') as fh:
     train_handler.load_optim_state(fh)
# this is needed to initialize `svi`
train_handler.fit(X_train, n_epochs=100, lr=0.001)

6367.41943359375

## Predict on training set and check fitted parameters

In [10]:
pred_handler = ModelHandler(model=predictive_model(train_handler.model_params), guide=guide)
pred_handler.optim_state = train_handler.optim_state 

In [13]:
preds_samples = pred_handler.predict(X_train, return_sites=[Site.days], num_samples=200)

In [14]:
latent_samples = train_handler.predict(X_train, return_sites=[Site.coefs, Site.coef_mus, Site.coef_sigmas], num_samples=200)

In [15]:
for site in [Site.coef_mus, Site.coef_sigmas]:
    samples_df = pd.DataFrame(latent_samples[site])
    samples_df.to_csv(f'../data/result/{site}.csv', index=False)

In [16]:
stats = summary(latent_samples, poisson=True)
df_edf = pd.read_csv('../data/preprocessed/edf.csv')
df_stats = stats_to_df(stats, df_edf.columns[2:-1])
df_stats.to_csv('../data/result/stats.csv', index=False)

In [18]:
preds = summary(preds_samples, poisson=False)
df_preds = preds_to_df(preds[Site.days])
df_preds.to_csv('../data/result/train_preds.csv', index=False)

## Predict on test set with only little data

In [19]:
X_test = jnp.array(np.load('../data/preprocessed/X_test.npz')['arr_0'])
X_test.shape

(115, 942, 24)

In [20]:
known_days = 7  # consider only known days of history
X_test_known = X_test[:, :known_days, :]

### Fit on known data

In [21]:
train_local_handler = ModelHandler(model=model, guide=local_guide(train_handler.model_params))

In [22]:
train_local_handler.fit(X_test_known, n_epochs=1_000, log_freq=200, lr=0.1)

epoch:    0 loss:          68.6854
epoch:  200 loss:          49.3261
epoch:  400 loss:          49.5041
epoch:  600 loss:          50.0771
epoch:  800 loss:          50.1159
epoch: 1000 loss:          50.4853


49.72303771972656

In [23]:
train_local_handler.fit(X_test_known, n_epochs=1_000, log_freq=200, lr=0.001)

epoch:    0 loss:          49.7230
epoch:  200 loss:          48.7056
epoch:  400 loss:          48.2238
epoch:  600 loss:          48.0156
epoch:  800 loss:          48.2500
epoch: 1000 loss:          48.5159


47.844234466552734

### Predict future of test data

In [24]:
params = train_handler.model_params
params.update(train_local_handler.model_params)
pred_local_handler = ModelHandler(model=predictive_model(params), guide=local_guide(params))
pred_local_handler.optim_state = train_local_handler.optim_state 

In [26]:
preds_samples = pred_local_handler.predict(X_test, return_sites=[Site.days], num_samples=200)

In [28]:
preds = summary(preds_samples, poisson=False)
df_preds = preds_to_df(preds[Site.days]).assign(StoreId=lambda df: df.StoreId + 1000)
df_preds.to_csv('../data/result/test_preds.csv', index=False)

### Compare with conventional Poisson regression using Scikit-Learn

In [29]:
reg = LinearRegression()

In [30]:
# select a single store_id
store_id = 16
X = np.nan_to_num(X_test_known, nan=1.0)[store_id, ...]
X, y = X[:, :-1], X[:, -1]

In [31]:
# we fit on the log-transformed target to achieve a multiplicate relationship
reg.fit(X, np.log(y))

LinearRegression()

In [32]:
# high overfit since we have more features than target values
np.exp(reg.predict(X)) - y

array([-1.8477440e-06, -7.8125000e-03,  8.7890625e-03,  9.7656250e-04,
        5.8593750e-03, -9.5367432e-07, -2.9296875e-03], dtype=float32)

In [34]:
# no overfitting in case of the Bayesian model
jnp.mean(preds_samples[Site.days], axis=0)[store_id][:known_days] - y

DeviceArray([5633.725   , -190.56982 , -947.0298  , -159.50977 ,
              -25.345215, 5612.475   ,  197.38965 ], dtype=float32)

### Compare the coefficients of conventional regression to the hierarchical model

In [35]:
# for many feature there is no meaningful value, i.e. 0, since they were not encountered in training
print(reg.coef_)

[ 1.1747563  -2.2386415   2.1442924   1.9889396   1.9385103   1.8024149
 -6.8102717   1.1747563   2.2386413  -2.2386413   0.          0.
 -0.09434899  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.        ]


In [36]:
# using the global prior it's possible to derive meaningful values
coefs_samples = pred_local_handler.predict(X_test_known, return_sites=[Site.coefs], num_samples=200)
print(jnp.mean(coefs_samples[Site.coefs], axis=0)[store_id])

[2.7901888  2.6591218  2.6881874  2.6126788  2.656775   2.554397
 2.4938385  0.33227605 3.115691   2.9264386  2.692092   2.9548435
 0.05613964 0.06542112 2.8379264  2.9023972  3.5701406  3.2074358
 4.0569873  2.9304545  2.7463424  2.823191   2.959007  ]


## Now compare those coefficients to the ones fitted on the whole time-series

In [37]:
all_local_handler = ModelHandler(model=model, guide=local_guide(train_handler.model_params))
all_local_handler.fit(X_test[store_id:store_id+1], n_epochs=10_000, log_freq=1_000, lr=0.001)

epoch:     0 loss:        7813.2896
epoch:  1000 loss:        7226.9590
epoch:  2000 loss:        7251.1680
epoch:  3000 loss:        7163.1885
epoch:  4000 loss:        7165.9688
epoch:  5000 loss:        7116.7900
epoch:  6000 loss:        7119.0649
epoch:  7000 loss:        7192.0601
epoch:  8000 loss:        7128.1733
epoch:  9000 loss:        7165.4888
epoch: 10000 loss:        7136.5723


7158.45068359375

In [38]:
# many coefficients are really similar but mind the log-space!
all_coefs_samples = all_local_handler.predict(X_test[store_id:store_id+1], return_sites=[Site.coefs], num_samples=200)
print(jnp.mean(all_coefs_samples[Site.coefs], axis=0)[0])

[ 2.814997    2.7551868   2.6182172   2.6453698   2.672692    2.5201027
  2.593036    0.2265296   3.184446    3.1163387   2.5429602   2.9477317
 -0.03218627  0.06836608  2.8726482   2.925492    3.56679     3.215817
  4.0523443   2.9164758   2.7241356   2.8247747   2.9598234 ]
